In [ ]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

lr_dgcnn = 0.01
weight_decay_dgcnn = 5e-4 #tem no codigo do GMSS tmbm
K=2
epochs=20 # DEFFERARD TMBM USA 20
device = 'cpu'

hiperparam_run = {
    'lr_dgcnn ':lr_dgcnn,
    'weight_decay_dgcnn':weight_decay_dgcnn,
    'K':K,
    'epochs':epochs
}

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 
#fn = 'de_LDS'
feature_list = ['de_LDS','psd_LDS', 'rasm_LDS', 'dasm_LDS', 'asm_LDS','dcau_LDS']

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")

all_acc_results = []

In [ ]:
# for cur_file in all_files:
cur_file = all_files[0]
# for fn in feature_list:
fn = 'dasm_LDS'
#item = "15_20130709.mat"
#item = "13_20140527.mat"
item = os.path.basename(cur_file)
cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

# cur_file = os.path.join(full_path, item)
all_data = sio.loadmat(cur_file)


train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)


In [ ]:
train_data.shape[1]

# Treino completo

In [153]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob
import itertools

# lr_dgcnn = 0.001#0.01
# weight_decay_dgcnn = 8e-5 # 5e-4tem no codigo do GMSS tmbm
#lr_dgcnn = 0.01#0.01
#weight_decay_dgcnn = 5e-4 # 5e-4tem no codigo do GMSS tmbm

K=2
epochs=20 # DEFFERARD TMBM USA 20
device = 'cpu'

# lr_dgcnn_list = [0.01,0.03, 0.05, 0.07, 0.09]
# weight_decay_dgcnn_list = [8e-5,5e-4,5e-4]
# out_channel_list = [32,48]

# # 0.0009,0.001,0.005,0.009,0.01
# lr_dgcnn_list = [0.009,0.005, 0.003, 0.001] #diminuindo.
# weight_decay_dgcnn_list = [5e-4,9e-4,0.001,0.005,0.009,0.01] #aumentando
# out_channel_list = [32,48]

# 0.0009,0.001,0.005,0.009,0.01
lr_dgcnn_list = [0.01] #diminuindo.
weight_decay_dgcnn_list = [5e-4] #aumentando
out_channel_list = [48]

all_hiperparams_combination = list(itertools.product(lr_dgcnn_list,weight_decay_dgcnn_list,out_channel_list))
# all_hiperparams_combination

hiperparam_list = []

for lr_dgcnn, weight_decay_dgcnn, out_channel_dgcnn in all_hiperparams_combination:
    cur_hiperparam_dict = {
        'lr_dgcnn':lr_dgcnn,
        'weight_decay_dgcnn':weight_decay_dgcnn,
        'K':K,
        'epochs':epochs,
        'out_channel_dgcnn':out_channel_dgcnn
        }
    hiperparam_list.append(cur_hiperparam_dict)

#hiperparam_list


label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 
#fn = 'de_LDS'
feature_list = ['de_LDS','psd_LDS', 'rasm_LDS', 'dasm_LDS', 'asm_LDS','dcau_LDS']

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")

all_acc_results = []
for hiperparam_run in hiperparam_list:
    lr_dgcnn = hiperparam_run['lr_dgcnn']
    weight_decay_dgcnn = hiperparam_run['weight_decay_dgcnn']
    out_channel_dgcnn = hiperparam_run['out_channel_dgcnn']

    for cur_file in all_files:
        for fn in feature_list:
            #item = "15_20130709.mat"
            #item = "13_20140527.mat"
            start_time = datetime.now()

            item = os.path.basename(cur_file)
            cur_user = item.split("_")[0]
            user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
            print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

            # cur_file = os.path.join(full_path, item)
            all_data = sio.loadmat(cur_file)


            train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)

            #train_data = preprocessing.scale(train_data)
            #test_data = preprocessing.scale(test_data)

            train_data_tensor = torch.from_numpy(train_data).to(torch.float)#.to(device)
            test_data_tensor = torch.from_numpy(test_data).to(torch.float)#.to(device)
            train_label_tensor = torch.from_numpy(train_label).to(torch.long)#.to(device)
            test_label_tensor = torch.from_numpy(test_label).to(torch.long)#.to(device)

            # define model
            model = DGCNN(
                in_channels=5, #5 frequencias
                num_electrodes=train_data.shape[1], #depende de cada feature
                k_adj=K, #numero de layers - copiando do GMSS - na verdade no GMSS K=ordem de chebyshev
                # out_channels=32, #copiando do GMSS, mas ainda nao entendi
                out_channels=out_channel_dgcnn,
                num_classes=3
            )

            best_test_res = {
                'acc':0,
                'predict_label':None,
                'trur_label':None
            }

            optimizer = optim.Adam(model.parameters(),
                                lr=lr_dgcnn, weight_decay=weight_decay_dgcnn)
            myloss = nn.CrossEntropyLoss()
            #print("matriz de adjacencia inicial:")
            #print(model.A)
            for cur_epoch in range(epochs):
                # ordem 1
                # model.train() # como apontar  referencia aos dados de treino? R: E na definicao do modelo
                # optimizer.zero_grad() #etapa comum em todos, zerar o gradiente.

                # output = model(train_data_tensor)
                # #TODO accuracy.
                # #print(output)

                # # import torch.nn.functional as F
                # # loss_train = F.nll_loss(output, train_label)

                # myloss = nn.CrossEntropyLoss()
                # loss = myloss(output, train_label_tensor)
                # loss.backward()
                # optimizer.step()

                # ordem 2
                model.train() # como apontar  referencia aos dados de treino? R: E na definicao do modelo
                output = model(train_data_tensor)
                # STEP 10 - Calculate the loss function
                loss = myloss(output, train_label_tensor)
                
                optimizer.zero_grad() #etapa comum em todos, zerar o gradiente.

                #TODO accuracy.
                #print(output)

                # import torch.nn.functional as F
                # loss_train = F.nll_loss(output, train_label)
                # STEP 11 - Updating the Adj matrix
                loss.backward()
                optimizer.step()
                # Estou sentindo falta do coidgo de atualizacao dessa matriz

                train_acc = torch.sum(torch.argmax(output, 1) == train_label_tensor) / train_data_tensor.shape[0]

                print('Epoch : {} -- TrainLoss : {} -- TrainAcc : {}\n'.format(cur_epoch, loss.cpu().data, train_acc.cpu().data))
                #print(f"matriz de adjacencia atualizada {cur_epoch}:")
                #print(model.A)
            #TODO print tempo de proc.
            # model.eval()
            test_output = model(test_data_tensor) # aqui eu preciso puxar a MAtriz de adjancencia do treino, nao comecar uma.
            test_loss = myloss(test_output, test_label_tensor)
            test_acc = torch.sum(torch.argmax(test_output, 1) == test_label_tensor) / test_data_tensor.shape[0]
            print('Epoch : {} -- TestLoss : {} -- TestAcc : {}\n'.format(cur_epoch, test_loss.cpu().data, test_acc.cpu().data))

            # save the best results    
            # if best_test_res['acc'] < test_acc.cpu().data:
            best_test_res['acc'] = test_acc.cpu().data 
            best_test_res['loss'] = test_loss.cpu().data 
            best_test_res['predict_label'] = torch.argmax(test_output, 0).cpu().numpy()
            best_test_res['true_label'] = test_label_tensor.cpu().numpy()
            print('update res')
            end_time = datetime.now()
            all_acc_results.append([end_time-start_time,fn,item,best_test_res, cur_file,hiperparam_run])

            print(best_test_res)
            # TODO logar em um arquivo os resultados totais.
            # salvar em TXT:
            filename_to_write = '20230917_1temp_analise_gcnn.txt'
            with open(filename_to_write, "a+") as f:
                rows_as_string = ';'.join([str(end_time-start_time),item,fn,str(hiperparam_run),str(best_test_res)])
                f.write(f"{rows_as_string}\n")

            # break



- reading file: 10_20131130.mat
--user:10, sessions: 2013-11-30 00:00:00
	### TRAIN ###
		de_LDS1 - video 1 - (235, 62, 5) observacoes
		de_LDS2 - video 2 - (233, 62, 5) observacoes
		de_LDS3 - video 3 - (206, 62, 5) observacoes
		de_LDS4 - video 4 - (238, 62, 5) observacoes
		de_LDS5 - video 5 - (185, 62, 5) observacoes
		de_LDS6 - video 6 - (195, 62, 5) observacoes
		de_LDS7 - video 7 - (237, 62, 5) observacoes
		de_LDS8 - video 8 - (216, 62, 5) observacoes
		de_LDS9 - video 9 - (265, 62, 5) observacoes
	### TEST ###
		de_LDS10 - video 10 - (237, 62, 5) observacoes
		de_LDS11 - video 11 - (235, 62, 5) observacoes
		de_LDS12 - video 12 - (233, 62, 5) observacoes
		de_LDS13 - video 13 - (235, 62, 5) observacoes
		de_LDS14 - video 14 - (238, 62, 5) observacoes
		de_LDS15 - video 15 - (206, 62, 5) observacoes
Epoch : 0 -- TrainLoss : 1.2224657535552979 -- TrainAcc : 0.3154228925704956

Epoch : 1 -- TrainLoss : 1.7710696458816528 -- TrainAcc : 0.44626864790916443

Epoch : 2 -- TrainLoss :

In [154]:
all_acc_results

[[datetime.timedelta(seconds=17, microseconds=506059),
  'de_LDS',
  '10_20131130.mat',
  {'acc': tensor(0.6662),
   'predict_label': array([1332,  468, 1177]),
   'trur_label': None,
   'loss': tensor(1.2374),
   'true_label': array([2, 2, 2, ..., 0, 0, 0])},
  '/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures/10_20131130.mat',
  {'lr_dgcnn': 0.01,
   'weight_decay_dgcnn': 0.0005,
   'K': 2,
   'epochs': 20,
   'out_channel_dgcnn': 48}],
 [datetime.timedelta(seconds=17, microseconds=416044),
  'psd_LDS',
  '10_20131130.mat',
  {'acc': tensor(0.5730),
   'predict_label': array([1332,  466,  188]),
   'trur_label': None,
   'loss': tensor(5.0919),
   'true_label': array([2, 2, 2, ..., 0, 0, 0])},
  '/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures/10_20131130.mat',
  {'lr_dgcnn': 0.01,
   'weight_decay_dgcnn': 0.0005,
   'K': 2,
   'epochs': 20,
   'out_channel_dgcnn': 48}],
 [datetime.timedelta(seconds=9, microseconds=720834),
  'rasm

In [155]:
import pandas as pd

In [156]:
gcnn_result_df = pd.DataFrame(all_acc_results)
gcnn_result_df

,0,1,2,3,4,5
0,0 days 00:00:17.506059,de_LDS,10_20131130.mat,"{'acc': tensor(0.6662), 'predict_label': [1332...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
1,0 days 00:00:17.416044,psd_LDS,10_20131130.mat,"{'acc': tensor(0.5730), 'predict_label': [1332...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
2,0 days 00:00:09.720834,rasm_LDS,10_20131130.mat,"{'acc': tensor(0.5195), 'predict_label': [845,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
3,0 days 00:00:09.530864,dasm_LDS,10_20131130.mat,"{'acc': tensor(0.5014), 'predict_label': [835,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
4,0 days 00:00:17.857517,asm_LDS,10_20131130.mat,"{'acc': tensor(0.4595), 'predict_label': [845,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
...,...,...,...,...,...,...
265,0 days 00:00:03.754693,psd_LDS,9_20140704.mat,"{'acc': tensor(0.8382), 'predict_label': [1382...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
266,0 days 00:00:02.394425,rasm_LDS,9_20140704.mat,"{'acc': tensor(0.8085), 'predict_label': [895,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
267,0 days 00:00:02.328905,dasm_LDS,9_20140704.mat,"{'acc': tensor(0.7435), 'predict_label': [885,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."
268,0 days 00:00:03.594192,asm_LDS,9_20140704.mat,"{'acc': tensor(0.7868), 'predict_label': [886,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000..."


In [157]:
#pd.DataFrame(gcnn_result_df[2].to_list())

In [158]:
gcnn_result_df[["user", "session"]]  = pd.DataFrame(gcnn_result_df[2].str.split('_').to_list())
gcnn_result_df[['acc','predict_label','trur_label','loss','true_label']] = pd.DataFrame(gcnn_result_df[3].to_list())
gcnn_result_df

,0,1,2,3,4,5,user,session,acc,predict_label,trur_label,loss,true_label
0,0 days 00:00:17.506059,de_LDS,10_20131130.mat,"{'acc': tensor(0.6662), 'predict_label': [1332...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",10,20131130.mat,tensor(0.6662),"[1332, 468, 1177]",None,tensor(1.2374),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,0 days 00:00:17.416044,psd_LDS,10_20131130.mat,"{'acc': tensor(0.5730), 'predict_label': [1332...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",10,20131130.mat,tensor(0.5730),"[1332, 466, 188]",None,tensor(5.0919),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
2,0 days 00:00:09.720834,rasm_LDS,10_20131130.mat,"{'acc': tensor(0.5195), 'predict_label': [845,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",10,20131130.mat,tensor(0.5195),"[845, 647, 196]",None,tensor(1.2854),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
3,0 days 00:00:09.530864,dasm_LDS,10_20131130.mat,"{'acc': tensor(0.5014), 'predict_label': [835,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",10,20131130.mat,tensor(0.5014),"[835, 646, 200]",None,tensor(1.3856),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
4,0 days 00:00:17.857517,asm_LDS,10_20131130.mat,"{'acc': tensor(0.4595), 'predict_label': [845,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",10,20131130.mat,tensor(0.4595),"[845, 647, 199]",None,tensor(1.2010),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,0 days 00:00:03.754693,psd_LDS,9_20140704.mat,"{'acc': tensor(0.8382), 'predict_label': [1382...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",9,20140704.mat,tensor(0.8382),"[1382, 449, 1126]",None,tensor(0.3862),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
266,0 days 00:00:02.394425,rasm_LDS,9_20140704.mat,"{'acc': tensor(0.8085), 'predict_label': [895,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",9,20140704.mat,tensor(0.8085),"[895, 367, 1177]",None,tensor(0.7673),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
267,0 days 00:00:02.328905,dasm_LDS,9_20140704.mat,"{'acc': tensor(0.7435), 'predict_label': [885,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",9,20140704.mat,tensor(0.7435),"[885, 368, 1133]",None,tensor(0.7255),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."
268,0 days 00:00:03.594192,asm_LDS,9_20140704.mat,"{'acc': tensor(0.7868), 'predict_label': [886,...",/home/joaovmrf/Documents/mestrado/SEED/SEED_EE...,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",9,20140704.mat,tensor(0.7868),"[886, 465, 1177]",None,tensor(0.7390),"[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ..."


In [159]:
# gcnn_result_df.to_csv('20230917_todos_resultados.csv', sep=';')

In [160]:
gcnn_result_df['acc'] = gcnn_result_df['acc'].apply(lambda x: x.item())

In [ ]:
# gcnn_result_df[[0,'user','session','acc']]

In [161]:
gcnn_result_df[5] = gcnn_result_df[5].apply(lambda x: str(x))

## tempo de processamento

In [163]:
print(processing_time_df['sum'].mean())
print(processing_time_df['sum'].max())

0 days 00:07:17.442295500
0 days 00:09:46.323613


In [162]:
processing_time_df = gcnn_result_df[[0,5,1,'user','session','acc']].groupby([5,1]).agg({'acc':['mean','std'], 0:'sum'}).reset_index()
processing_time_df.columns = processing_time_df.columns.droplevel()
processing_time_df

,,,mean,std,sum
0,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,0.661031,0.165048,0 days 00:08:23.821132
1,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",dasm_LDS,0.643064,0.169067,0 days 00:05:33.747178
2,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",dcau_LDS,0.715189,0.152616,0 days 00:04:45.672837
3,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",de_LDS,0.766346,0.118181,0 days 00:09:30.582248
4,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",psd_LDS,0.610999,0.180598,0 days 00:09:46.323613
5,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",rasm_LDS,0.658960,0.172030,0 days 00:05:44.506765


## calculo das acuracias

In [178]:
# teste,pegar as duas maiores acuracias;
duas_maiores_acuracias = pd.DataFrame(gcnn_result_df[[5,1,'user','session','acc']]\
                                    #   .sort_values([5,'user', 'acc'], ascending=[True,False, False])\
    # .groupby([5,'user'])['acc']\
    .groupby([5,1,'user'])['acc']\
    .nlargest(2)\
        ).reset_index()
    #.to_list()
    #.agg({'acc':['mean','std']})
duas_maiores_acuracias

,5,1,user,level_3,acc
0,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,1,112,0.799855
1,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,1,124,0.662572
2,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,10,10,0.731214
3,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,10,16,0.468931
4,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,11,34,0.815029
...,...,...,...,...,...
175,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",rasm_LDS,7,218,0.631503
176,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",rasm_LDS,8,248,0.798410
177,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",rasm_LDS,8,236,0.776734
178,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",rasm_LDS,9,254,0.835983


In [180]:
cur_row_to_insert = duas_maiores_acuracias.groupby([5,1]).agg({'acc':['mean','std']}).reset_index()
cur_row_to_insert.columns = cur_row_to_insert.columns.droplevel()
cur_row_to_insert.sort_values(['mean'], ascending=[False])

,,,mean,std
3,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",de_LDS,0.811633,0.099853
2,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",dcau_LDS,0.777071,0.120301
5,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",rasm_LDS,0.736874,0.134120
0,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",asm_LDS,0.729793,0.132427
1,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",dasm_LDS,0.721243,0.134290
4,"{'lr_dgcnn': 0.01, 'weight_decay_dgcnn': 0.000...",psd_LDS,0.684827,0.145204


In [ ]:
row_with_info = pd.DataFrame()

In [ ]:
# duas_maiores_acuracias.agg({'acc':['mean','std']}).reset_index()
# duas_maiores_acuracias.columns = ['_'.join(col) for col in duas_maiores_acuracias.columns]
# duas_maiores_acuracias.columns
row_with_info = row_with_info.append(cur_row_to_insert)
row_with_info.loc[:,'hiperparams'] = str(hiperparam_run)
row_with_info.loc[:,'technic'] = '2 maiores'
row_with_info.loc[:,'processing_time'] = ''

row_with_info

In [ ]:
# acuracia_para_todos.values.flatten()
# acuracia_para_todos.loc[:,'hiperparams'] = str(hiperparam_run)
# acuracia_para_todos.loc[:,'technic'] = 'todas as sessoes'
# acuracia_para_todos.loc[:,'processing_time'] = ''
# acuracia_para_todos

In [ ]:
acuracia_para_todos = gcnn_result_df[[5,'user','session','acc']].sort_values([5,'user', 'acc'], ascending=[True,False, False])\
.groupby([5]).agg({'acc':['mean','std']}).reset_index()
acuracia_para_todos.columns = acuracia_para_todos.columns.droplevel(0)
acuracia_para_todos.loc[:,'hiperparams'] = str(hiperparam_run)
acuracia_para_todos.loc[:,'technic'] = 'todas as sessoes'
acuracia_para_todos.loc[:,'processing_time'] = ''

cur_row_to_insert = acuracia_para_todos 

row_with_info = row_with_info.append([cur_row_to_insert])
row_with_info['data_da_rodada'] = datetime.now()

row_with_info.sort_values(['mean'], ascending=[False])


In [ ]:
row_with_info.sort_values(['mean'], ascending=[False])

In [ ]:
# salvar em TXT:
# filename_to_write = 'analise_gcnn.csv'
# with open(filename_to_write, "a+") as f:
#     rows_as_string = row_with_info.to_string(header=True, index=True)
#     f.write(rows_as_string)

filename_to_write = './20230917_1analise_gcnn.csv'
row_with_info.sort_values(['mean'], ascending=[False]).to_csv(filename_to_write,mode='a',header=True, index=True, sep=';')


In [ ]:
# eda_gcnn = gcnn_result_df[['user', 'session','acc', 'predict_label','trur_label','loss','true_label']].sort_values(['user', 'acc'], ascending=[False, False])
eda_gcnn = gcnn_result_df[['user', 0,'session','acc']].sort_values(['user', 'acc'], ascending=[False, False]).to_csv(f"GCNN_WITHOUT_EVAL_20230912.csv")

eda_gcnn

In [ ]:
eda_gcnn.reset_index().to_csv(f"GCNN_WITHOUT_EVAL_20230912.csv")


In [ ]:
eda_gcnn.groupby(['user', 0,'session']).max()#.acc.mean()

In [ ]:
eda_gcnn.groupby(['user']).head(2).reset_index(drop=True)#.acc.mean()

# analisar dataset

In [ ]:
import glob
import os
from datetime import datetime

import numpy as np
import scipy.io as sio
import torch
import torch.nn as nn
import torch.optim as optim
from model import DGCNN
from sklearn import preprocessing
from utils import generating_data
from glob import glob

In [ ]:
fn = 'de_LDS'

label_list = np.array([1,0,-1,-1,0,1,-1,0,1,1,0,-1,0,1,-1]) + 1 # -1 for 

full_path = "/home/joaovmrf/Documents/mestrado/SEED/SEED_EEG/SEED_EEG/ExtractedFeatures"
all_files = [each_file for each_file in glob(full_path+'/*.mat') if 'label.mat' not in each_file]
all_files.sort()
# all_files = glob.glob(full_path+"*")


# for cur_file in all_files: 
#item = "15_20130709.mat"
#item = "13_20140527.mat"
cur_file = all_files[10]
item = os.path.basename(cur_file)
cur_user = item.split("_")[0]
user_session = datetime.strptime(item.split("_")[1][:-4],"%Y%m%d")
print(f"- reading file: {item}\n--user:{cur_user}, sessions: {user_session}")

# cur_file = os.path.join(full_path, item)
all_data = sio.loadmat(cur_file)


train_data, test_data, train_label, test_label = generating_data(all_data, label_list, fn)

In [ ]:
test_data.size

In [ ]:
test_data_tensor = torch.from_numpy(test_data).to(torch.float)#.to(device)
test_data_tensor.size()

In [ ]:
# pegando apenas o primeiro video (236), todos os canais, a ultima frequencia
test_data_tensor[:236,:,4]

In [ ]:
# pegando apenas o primeiro video (236), canal 1, frequencia delta
test_data_tensor[:236,0,0].size()

# Mudando eixo para entneder matriz de features

In [ ]:
test_data.shape

In [ ]:
test_data_swapped = np.swapaxes(test_data, 0, 1)
test_data_swapped = np.swapaxes(test_data_swapped, 1, 2)
test_data_swapped.shape

In [ ]:
test_data_swapped_tensor = torch.from_numpy(test_data_swapped).to(torch.float)#.to(device)


In [ ]:
test_data_swapped_tensor.size()

In [ ]:
# pegando todos os 62 canais, apenas frequencia delta 
test_data_swapped_tensor[:,0,:].size()

# Montar dicionario com parametros de treino;

In [ ]:
import itertools
lr_dgcnn_list = [0.01,0.03, 0.05, 0.07, 0.09]
weight_decay_dgcnn_list = [8e-5,5e-4,5e-4]
out_channel_list = [32,48]

In [ ]:
import itertools
# lr_dgcnn_list = [0.01,0.03, 0.05, 0.07, 0.09]
# weight_decay_dgcnn_list = [8e-5,5e-4,5e-4]
# out_channel_list = [32,48]

# lr_dgcnn_list = [0.009,0.005, 0.003, 0.001] #diminuindo.
# weight_decay_dgcnn_list = [5e-4,9e-4,0.001,0.005,0.009,0.01] #aumentando
# out_channel_list = [32,48]


lr_dgcnn_list = [0.01] #diminuindo.
weight_decay_dgcnn_list = [5e-4] #aumentando
out_channel_list = [48]

all_hiperparams_combination = list(itertools.product(lr_dgcnn_list,weight_decay_dgcnn_list,out_channel_list))
# all_hiperparams_combination

hiperparam_list = []

for lr_dgcnn, weight_decay_dgcnn, out_channel in all_hiperparams_combination:
    cur_hiperparam_dict = {
        'lr_dgcnn ':lr_dgcnn,
        'weight_decay_dgcnn':weight_decay_dgcnn,
        'K':K,
        'epochs':epochs,
        'out_channel':out_channel
        }
    hiperparam_list.append(cur_hiperparam_dict)

# hiperparam_list
all_hiperparams_combination

In [ ]:
hiperparam_list = []

for lr_dgcnn, weight_decay_dgcnn, out_channel in all_hiperparams_combination:
    cur_hiperparam_dict = {
        'lr_dgcnn ':lr_dgcnn,
        'weight_decay_dgcnn':weight_decay_dgcnn,
        'K':K,
        'epochs':epochs,
        'out_channel':out_channel
        }
    hiperparam_list.append(cur_hiperparam_dict)

hiperparam_list


# Teste de escrita

In [ ]:
filename_to_write = 'teste.txt'
with open(filename_to_write, "a+") as f:
    rows_as_string =';'.join(['oi', 'teste', '20e22'])
    f.write(f"{rows_as_string}\n")

# EDA DE hiperparams

In [ ]:
eda_de_hiperparams = pd.read_csv("temp_analise_gcnn.txt", header=None, delimiter=';')
eda_de_hiperparams.shape

In [ ]:
eda_de_hiperparams[["user", "session"]]  = pd.DataFrame(eda_de_hiperparams[1].str.split('_').to_list())


In [ ]:
eda_de_hiperparams

In [ ]:
# eda_de_hiperparams[['acc','predict_label','trur_label','loss','true_label']] = 
# import json
# json.loads(eda_de_hiperparams.loc[0,4])

#import ast
# ast.literal_eval(eda_de_hiperparams.loc[0,4])
# ast.literal_eval(eda_de_hiperparams.loc[0,4].replace("'",'"'))

#eval(eda_de_hiperparams.loc[0,4].replace("'",'"').replace('tensor', '').replace('array',''))
# eda_de_hiperparams.loc[0,4]

In [ ]:
# eda_de_hiperparams[['acc','predict_label','trur_label','loss','true_label']] 
eda_de_hiperparams['4_as_dict'] = eda_de_hiperparams[4].apply(lambda x: eval(x.replace("'",'"').replace('tensor', '').replace('array','')))
eda_de_hiperparams[['acc','predict_label','trur_label','loss','true_label']] = pd.DataFrame(eda_de_hiperparams['4_as_dict'].to_list())
# eda_de_hiperparams

In [ ]:
eda_de_hiperparams[[2,'user','session', 3, 4,'acc']]

In [ ]:
duas_maiores_acc_all = eda_de_hiperparams[[2,'user','session', 3, 4,'acc']]\
    .groupby([3, 'user'])['acc']\
    .nlargest(2)\
    .reset_index()
duas_maiores_acc_all

In [ ]:
duas_maiores_acc_all_agg = duas_maiores_acc_all.groupby([3]).agg({'acc':['mean','std']}).reset_index()\

duas_maiores_acc_all_agg.columns = duas_maiores_acc_all_agg.columns.droplevel()
duas_maiores_acc_all_agg.sort_values(['mean'], ascending=[False])\
    # .to_csv("long_result_with_3.csv",sep=';')

In [ ]:
duas_maiores_acc_all_agg.loc[30,:]

In [ ]:
lr_dgcnn_list = [0.01,0.03, 0.05, 0.07, 0.09]
weight_decay_dgcnn_list = [8e-5,5e-4]
out_channel_list = [32,48]

all_hiperparams_combination = list(itertools.product(lr_dgcnn_list,weight_decay_dgcnn_list,out_channel_list))
all_hiperparams_combination

In [ ]:
8e-5